
# Jeanette's Workbook

<hr style="border:2px solid red"> </hr>


### Detecting Anomalies on the Codeup Curriculum Website

We recieved a fake email from a staff member at Codeup:
  
  
Hello,
  
  
I have some questions for you that I need to be answered before the Board meeting on Monday evening. I need to be able to speak to the following questions. I also need a single slide that I can incorporate into my existing presentation (Google Slides) that summarizes the most important points. My questions are listed below; however, if you discover anything else important that I didn’t think to ask, please include that as well.
  
  
1. Which lesson appears to attract the most traffic consistently across cohorts (per program)?
2. Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?
3. Are there students who, when active, hardly access the curriculum? If so, what information do you have about these students?
4. Is there any suspicious activity, such as users/machines/etc accessing the curriculum who shouldn’t be? Does it appear that any web-scraping is happening? Are there any suspicious IP addresses?
5. At some point in 2019, the ability for students and alumni to access both curriculums (web dev to ds, ds to web dev) should have been shut off. Do you see any evidence of that happening? Did it happen before?
6. What topics are grads continuing to reference after graduation and into their jobs (for each program)?
7. Which lessons are least accessed?
8. Anything else I should be aware of?
  
  
Thank you,
  
Mr. Codeup

#### My responsibilities for this project will be the README, the Presentation slide, and questions [2, 4, 5, 8].


In [3]:
# Data Science Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# DBSCAN import
from sklearn.cluster import DBSCAN

# Scaler import
from sklearn.preprocessing import MinMaxScaler

import wrangle

import warnings
warnings.filterwarnings("ignore")

In [5]:
df = wrangle.acquire_logs()

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 847330 entries, 0 to 847329
Data columns (total 9 columns):
 #   Column      Non-Null Count   Dtype 
---  ------      --------------   ----- 
 0   date        847330 non-null  object
 1   time        847330 non-null  object
 2   path        847329 non-null  object
 3   ip          847330 non-null  object
 4   user_id     847330 non-null  int64 
 5   name        847330 non-null  object
 6   program_id  847330 non-null  int64 
 7   start_date  847330 non-null  object
 8   end_date    847330 non-null  object
dtypes: int64(2), object(7)
memory usage: 64.6+ MB


In [2]:
df = wrangle.wrangle_logs()
df.head()

,path,ip,user_id,name,program_id,start_date,end_date,program_name
2018-01-26 09:55:03,/,97.105.19.61,1,Hampton,1,2015-09-22,2016-02-06,WebDev-PHP
2018-01-26 09:56:02,java-ii,97.105.19.61,1,Hampton,1,2015-09-22,2016-02-06,WebDev-PHP
2018-01-26 09:56:05,java-ii/object-oriented-programming,97.105.19.61,1,Hampton,1,2015-09-22,2016-02-06,WebDev-PHP
2018-01-26 09:56:06,slides/object_oriented_programming,97.105.19.61,1,Hampton,1,2015-09-22,2016-02-06,WebDev-PHP
2018-01-26 09:56:24,javascript-i/conditionals,97.105.19.61,2,Teddy,2,2018-01-08,2018-05-17,WebDev-Java


In [7]:
df.program_id.value_counts()

2    713365
3    103412
1     30548
4         5
Name: program_id, dtype: int64

In [8]:
df.user_id.value_counts()

11     17913
64     16322
53     12329
314     7783
1       7404
       ...  
879        1
940        1
163        1
165        1
652        1
Name: user_id, Length: 911, dtype: int64

The first question I will try to answer is:
## 2. Is there a cohort that referred to a lesson significantly more than other cohorts seemed to gloss over?

Additional thoughts:
- What is a lesson? I will assume it is a page that is not a supporting page (like overview or index). I will assume it is a non-supportive page within a module.

In [9]:
# How many cohorts does this dataset have?
print(df.name.value_counts().count())
df.name.value_counts()

47


Staff         84031
Ceres         40730
Zion          38096
Jupiter       37109
Fortuna       36902
Voyageurs     35636
Ganymede      33844
Apex          33568
Deimos        32888
Darden        32015
Teddy         30926
Hyperion      29855
Betelgeuse    29356
Ulysses       28534
Europa        28033
Xanadu        27749
Bayes         26538
Wrangell      25586
Andromeda     25359
Kalypso       23691
Curie         21582
Yosemite      20743
Bash          17713
Luna          16623
Marco         16397
Easley        14715
Lassen         9587
Arches         8890
Florence       8562
Sequoia        7444
Neptune        7276
Olympic        4954
Kings          2845
Pinnacles      2158
Hampton        1712
Oberon         1672
Quincy         1237
Niagara         755
Mammoth         691
Glacier         598
Joshua          302
Ike             253
Badlands         93
Franklin         72
Apollo            5
Denali            4
Everglades        1
Name: name, dtype: int64

In [11]:
# Let's see pages viewed and how many of those were unqiue pages
page_views = df.groupby(['name'])['path'].agg(['count', 'nunique'])
page_views

,count,nunique
name,,
Andromeda,25359,366
Apex,33568,273
Apollo,5,4
Arches,8890,342
Badlands,93,44
Bash,17713,275
Bayes,26538,533
Betelgeuse,29356,334
Ceres,40730,301


In [18]:

# applying groupby() function to
# group the data on team value.
a = df.groupby('path')
  
# Let's print the first entries
# in all the groups formed.
a.first()

,ip,user_id,name,program_id,start_date,end_date,program_name
path,,,,,,,
%20https://github.com/RaulCPena,74.192.167.159,580,Curie,3,2020-02-03,2020-07-07,DataSci
',71.150.217.33,64,Staff,2,2014-02-04,2014-02-04,WebDev-Java
",%20https://github.com/RaulCPena",74.192.167.159,580,Curie,3,2020-02-03,2020-07-07,DataSci
.git,72.178.123.71,203,Voyageurs,2,2018-05-29,2018-10-11,WebDev-Java
.gitignore,72.178.123.71,203,Voyageurs,2,2018-05-29,2018-10-11,WebDev-Java
...,...,...,...,...,...,...,...
web-dev-day-two,99.43.143.116,700,Bash,2,2020-07-20,2021-01-21,WebDev-Java
where,136.49.16.164,949,Florence,3,2021-03-15,2021-09-03,DataSci
working-with-time-series-data,96.8.130.213,692,Darden,3,2020-07-13,2021-01-12,DataSci


In [20]:

# applying groupby() function to
# group the data on team value.
b = df.groupby('path').agg('count')
  
# Let's print the first entries
# in all the groups formed.
b

,ip,user_id,name,program_id,start_date,end_date,program_name
path,,,,,,,
%20https://github.com/RaulCPena,1,1,1,1,1,1,1
',1,1,1,1,1,1,1
",%20https://github.com/RaulCPena",1,1,1,1,1,1,1
.git,1,1,1,1,1,1,1
.gitignore,2,2,2,2,2,2,2
...,...,...,...,...,...,...,...
web-dev-day-two,2,2,2,2,2,2,2
where,1,1,1,1,1,1,1
working-with-time-series-data,15,15,15,15,15,15,15
